In [1]:
import pandas as pd
import requests
from datetime import datetime, timezone
import time

In [2]:
def get_trades(pair, startDate, endDate):
    endpoint = 'https://api.kraken.com/0/public/Trades'
    
    startTime = int(datetime.strptime(startDate, '%Y-%m-%d').replace(tzinfo=timezone.utc).timestamp()*1000000000)
    endTime = int(datetime.strptime(endDate, '%Y-%m-%d').replace(tzinfo=timezone.utc).timestamp()*1000000000)
    
    last_time = startTime
    df_result = pd.DataFrame()
    
    while last_time < endTime:
        parameters = {'pair': pair,'since': last_time}
        response = requests.get(url=endpoint, params=parameters)
        try:
            data = response.json()['result'][pair]
            last_time = int(response.json()['result']["last"])
            
        except ValueError:
            return df_result
        
        df_data = pd.DataFrame.from_records(data,
                             columns=['Price', 'Volume', 'Time', 'BuySell', 'MarketLimit', 'Misc'])
        df_data['Time'] = pd.to_datetime(df_data['Time'], unit='s')
        df_result = df_result.append(df_data)
        time.sleep(2)
    
    try:
        df_result.set_index("Time", inplace = True)    
        df_result = df_result.loc[startDate:endDate+' 00:00:00']
    except ValueError:
        return df_result
    
    return df_result

Kraken Public(REST Market Data) - 
The public endpoints are rate limited by IP address and currency pair for calls to Trades and OHLC, and by IP address only for calls to all other public endpoints.
Calling the public endpoints at a frequency of 1 per second (or less) would remain within the rate limits, but exceeding this frequency could cause the calls to be rate limited.

In [3]:
df_ADA_USD = get_trades('ADAUSD','2021-07-01','2021-08-23')

In [5]:
df_ADA_USD

,Price,Volume,BuySell,MarketLimit,Misc
Time,,,,,
2021-02-01 00:00:21.149600000,0.345201,55.00000000,b,m,
2021-02-01 00:00:23.852900096,0.345200,1830.76400003,s,l,
2021-02-01 00:00:30.689299968,0.345000,3000.00000000,s,l,
2021-02-01 00:00:30.692500224,0.345000,367.75463768,s,l,
2021-02-01 00:01:22.767899904,0.345038,1112.10000000,b,m,
...,...,...,...,...,...
2021-08-22 23:59:53.022000128,2.710422,5000.00000000,s,m,
2021-08-22 23:59:53.023500032,2.710422,500.00000000,s,m,
2021-08-22 23:59:58.372300032,2.710000,500.00000000,s,m,


In [6]:
df_ADA_USD.to_csv('ADAUSD_trades_data.csv')